## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb　←いまここ
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

## ここから本編

In [44]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import re

研究課題の主な項目をデータフレームにして、pickleして保存する関数を定義する。

In [45]:
def kadai(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    kadailist = []
    for grantAward in tree.iterfind("grantAward"):
        projecttype = grantAward.get("projectType")
        awardnumber = grantAward.get("awardNumber")
        startfiscalyear = grantAward.find("summary[@xml:lang='ja']/periodOfAward", nsmap).get("searchStartFiscalYear")
        endfiscalyear = grantAward.find("summary[@xml:lang='ja']/periodOfAward", nsmap).get("searchEndFiscalYear")

        try:
            category_niicode = grantAward.find("summary[@xml:lang='ja']/category", nsmap).get("niiCode")
        except:
            category_niicode = np.NaN
        
        try:
            category = grantAward.find("summary[@xml:lang='ja']/category", nsmap).text
        except:
            category = np.NaN
            
        try:
            section_niicode = grantAward.find("summary[@xml:lang='ja']/section", nsmap).get("niiCode")
        except:
            section_niicode = np.NaN
        
        try:
            section = grantAward.find("summary[@xml:lang='ja']/section", nsmap).text
        except:
            section = np.NaN

        try:
            title_ja = grantAward.find("summary[@xml:lang='ja']/title", nsmap).text
        except:
            title_ja = np.NaN
        
        try:
            title_en = grantAward.find("summary[@xml:lang='en']/title", nsmap).text
        except:
            title_en = np.NaN
        
        try:
            directcost = grantAward.find("summary[@xml:lang='ja']/overallAwardAmount/directCost", nsmap).text
        except:
            directcost = np.NaN

        row = [
            awardnumber,
            projecttype,
            category,
            category_niicode,
            section,
            section_niicode,
            startfiscalyear,
            endfiscalyear,
            directcost,
            title_ja,
            title_en,
        ]
        kadailist.append(row)
        
    df = pd.DataFrame(kadailist)
    df.columns = [
        'awardnumber',
        'projecttype',
        'category',
        'category_niicode',
        'section',
        'section_niicode',
        'startfiscalyear',
        'endfiscalyear',
        'directcost',
        'title_ja',
        'title_en',
    ]
    
    pickledfile = 'pickledDF_grantaward_main/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
    df.to_pickle(pickledfile)

フォルダをいったんきれいにする関数を定義して、実行する

In [47]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_grantaward_main')

2010年以降のxmlファイルをパースして、1個ずつpickleして保存

In [48]:
for xmlfile in tqdm(glob.glob('xml/201*.xml')):
    kadai(xmlfile)

空のデータフレームを作って…

In [49]:
columns = [
    'awardnumber',
    'projecttype',
    'category',
    'category_niicode',
    'section',
    'section_niicode',
    'startfiscalyear',
    'endfiscalyear',
    'directcost',
    'title_ja',
    'title_en',
]
df = pd.DataFrame(columns=columns)

pickleからデータフレームを復元、すべて結合

In [50]:
for dump in tqdm(glob.glob('pickledDF_grantaward_main/201*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [51]:
df.to_pickle('beforeCleaning/parse_grantaward_main.dump')

### データクリーニング

pickleからデータフレームを復元する

In [52]:
with open('beforeCleaning/parse_grantaward_main.dump', mode='rb') as f:
    df = pickle.load(f)

データを概観する

In [53]:
df

,awardnumber,projecttype,category,category_niicode,section,section_niicode,startfiscalyear,endfiscalyear,directcost,title_ja,title_en
0,S003,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,包括型脳科学研究推進支援ネットワーク,NaN
1,S002,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,ゲノム科学の総合的推進に向けた大規模ゲノム情報生産・高度情報解析支援,NaN
2,S001,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,がん研究分野の特性等を踏まえた支援活動,NaN
3,4203,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,統合的多階層生体機能学領域の確立とその応用,Establishment of Integrative Multi-level Syste...
4,4202,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,質感認知の脳神経メカニズムと高度質感情報処理技術の融合的研究,Integrative studies of neural mechanisms and a...
5,4201,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,システム的統合理解に基づくがんの先端的診断、治療、予防法の開発,Integrative Systems Understanding of Cancer fo...
6,3224,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,先端技術を駆使したＨＬＡ多型・進化・疾病に関する統合的研究,"HLA polymorphism, disease and evolution"
7,3223,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,性差構築の分子基盤,Molecular mechanisms for establishment of sex ...
8,3222,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,ゲノム複製・修復・転写のカップリングと普遍的なクロマチン構造変換機構,"Coupling of replication, repair and transcript..."
9,3221,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,癌幹細胞を標的とする腫瘍根絶技術の新構築,Development of Novel Treatment Strategies Targ...


重複したデータがあるかどうか

In [54]:
df.duplicated().any()

False

重複はなかった（あったら何かが違う）。

awardnumberがユニークかどうか確認するために、行数255813とawardnumberのユニークな値の数が一致するかどうか確認する。

In [55]:
df['awardnumber'].nunique(dropna=False)

258813

awardnumberはユニークだったので（ユニークじゃなかったら何かが違う）、インデックスにする。

In [56]:
df = df.set_index('awardnumber')
df

,projecttype,category,category_niicode,section,section_niicode,startfiscalyear,endfiscalyear,directcost,title_ja,title_en
awardnumber,,,,,,,,,,
S003,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,包括型脳科学研究推進支援ネットワーク,NaN
S002,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,ゲノム科学の総合的推進に向けた大規模ゲノム情報生産・高度情報解析支援,NaN
S001,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2015,NaN,がん研究分野の特性等を踏まえた支援活動,NaN
4203,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,統合的多階層生体機能学領域の確立とその応用,Establishment of Integrative Multi-level Syste...
4202,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,質感認知の脳神経メカニズムと高度質感情報処理技術の融合的研究,Integrative studies of neural mechanisms and a...
4201,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,システム的統合理解に基づくがんの先端的診断、治療、予防法の開発,Integrative Systems Understanding of Cancer fo...
3224,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,先端技術を駆使したＨＬＡ多型・進化・疾病に関する統合的研究,"HLA polymorphism, disease and evolution"
3223,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,性差構築の分子基盤,Molecular mechanisms for establishment of sex ...
3222,area,新学術領域研究(研究領域提案型),73,NaN,NaN,2010,2014,NaN,ゲノム複製・修復・転写のカップリングと普遍的なクロマチン構造変換機構,"Coupling of replication, repair and transcript..."


構造を見ておく

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 10 columns):
projecttype         258813 non-null object
category            258804 non-null object
category_niicode    258804 non-null object
section             150369 non-null object
section_niicode     150369 non-null object
startfiscalyear     258813 non-null object
endfiscalyear       258813 non-null object
directcost          258611 non-null object
title_ja            258668 non-null object
title_en            116672 non-null object
dtypes: object(10)
memory usage: 21.7+ MB


nullを0に置換しておく。

In [58]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 10 columns):
projecttype         258813 non-null object
category            258813 non-null object
category_niicode    258813 non-null object
section             258813 non-null object
section_niicode     258813 non-null object
startfiscalyear     258813 non-null object
endfiscalyear       258813 non-null object
directcost          258813 non-null object
title_ja            258813 non-null object
title_en            258813 non-null object
dtypes: object(10)
memory usage: 21.7+ MB


データ型を指定しておく。

In [59]:
df = df.astype({
    'category_niicode': np.int64,
    'section_niicode': np.int64,
    'startfiscalyear': np.int64,
    'endfiscalyear': np.int64,
    'directcost': np.int64,
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258813 entries, S003 to 18K08432
Data columns (total 10 columns):
projecttype         258813 non-null object
category            258813 non-null object
category_niicode    258813 non-null int64
section             258813 non-null object
section_niicode     258813 non-null int64
startfiscalyear     258813 non-null int64
endfiscalyear       258813 non-null int64
directcost          258813 non-null int64
title_ja            258813 non-null object
title_en            258813 non-null object
dtypes: int64(5), object(5)
memory usage: 21.7+ MB


directcostを円単位から、千円単位にする。

In [60]:
df['directcost'] = df['directcost'].map(lambda x: x // 1000)

niicodeを精査する

In [61]:
pd.crosstab(df['category_niicode'], df['startfiscalyear'], margins=True)

startfiscalyear,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
category_niicode,,,,,,,,,,
0,3,5,1,0,0,0,0,0,0,9
28,10,1,16,36,14,14,14,13,12,130
42,3040,2755,2871,2912,2819,2691,2694,2543,2308,24633
55,0,0,0,1,4,3,3,4,0,15
60,89,91,88,87,87,87,95,82,0,706
62,256,70,0,0,0,0,0,0,0,326
63,714,697,706,707,708,708,736,725,557,6258
64,347,505,399,394,408,390,423,433,0,3299
65,5856,6739,6286,6079,5886,5745,5735,5817,0,48143


niicodeが0になっているものが9件あった。まず9件のデータを見る。

In [62]:
df[df['category_niicode'] == 0]

,projecttype,category,category_niicode,section,section_niicode,startfiscalyear,endfiscalyear,directcost,title_ja,title_en
awardnumber,,,,,,,,,,
22128009,project,0,0,0,0,2010,2014,0,悉皆的二次代謝経路推定に向けたデータベースおよび要素技術の研究開発,0
22900002,project,0,0,0,0,2010,2010,0,2011年東北地方太平洋沖地震に関する総合調査,Comprehensive studies for the 2011 off the Pac...
22900001,project,0,0,0,0,2010,2011,0,2011年霧島火山(新燃岳)噴火に関する総合調査,General study on the eruption at Shinmoedake v...
11F01767,project,0,0,0,0,2011,2011,0,冷陽電子の大量蓄積法の開発と冷反水素原子の生成・分光,0
11F01514,project,0,0,0,0,2011,2013,0,新規スプライシング制御因子GomafuとMalat1生理機能の解析,0
11F01303,project,0,0,0,0,2011,2013,0,社会恐怖の医学思想史に向けて―20世紀の日仏比較研究,0
23900002,project,0,0,0,0,2011,2011,0,2011年東北地方太平洋沖地震に関する総合調査,Comprehensive studies for the 2011 off the Pac...
23900001,project,0,0,0,0,2011,2012,0,学術振興施策に資するための大学への投資効果等に関する調査研究,Research about the Effect of Investment to Uni...
24900001,project,0,0,0,0,2012,2012,0,平成24年5月6日に北関東で発生した竜巻の発生メカニズムと被害実態の総合調査,A comprehensive survey on mechanism of tornado...


タイトルなどをgoogle検索したところ、種目が判明した。category_niicodeを代入しておく。

In [63]:
df.loc['22128009', 'category_niicode'] = 73 #  新学術領域研究(研究領域提案型)
df.loc['22900002', 'category_niicode'] = 55 #  特別研究推進費
df.loc['22900001', 'category_niicode'] = 55 #  特別研究推進費
df.loc['11F01767', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['11F01514', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['11F01303', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['23900002', 'category_niicode'] = 55 #  特別研究推進費
df.loc['23900001', 'category_niicode'] = 55 #  特別研究推進費
df.loc['24900001', 'category_niicode'] = 55 #  特別研究推進費

category_niicodeが全部決まったことを確認する。

In [64]:
pd.crosstab(df['category_niicode'], df['startfiscalyear'], margins=True)

startfiscalyear,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
category_niicode,,,,,,,,,,
28,10,1,16,36,14,14,14,13,12,130
42,3040,2758,2871,2912,2819,2691,2694,2543,2308,24636
55,2,2,1,1,4,3,3,4,0,20
60,89,91,88,87,87,87,95,82,0,706
62,256,70,0,0,0,0,0,0,0,326
63,714,697,706,707,708,708,736,725,557,6258
64,347,505,399,394,408,390,423,433,0,3299
65,5856,6739,6286,6079,5886,5745,5735,5817,0,48143
68,576,710,641,691,632,597,634,636,605,5722


クリーニング後の課題データフレームをpickleして保存する

In [65]:
df.to_pickle('afterCleaning/parse_kadai.dump')

ひとまず終了。

次は、parse_institution.ipynbで内定時の研究機関を取得する。